# ATENÇÃO

**Antes de ir usar as funções, certifique-se de rodas as células que as definem, bem como a célula que importa as bibliotecas e os módulos necessários.**

Script elaborado por Vinícius de Almeida Nery Ferreira a partir de dados do Ministério da Saúde. Para mais informações e estatísticas acerca do Coronavírus, visite: https://covid.saude.gov.br


In [6]:
import pandas as pd 
import numpy as np 
from IPython.display import clear_output
import os
import glob
import pathlib
import plotly
import plotly.offline as py
from plotly.subplots import make_subplots
from plotly.offline import plot,iplot
plotly.offline.init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'vscode'
#pio.renderers.default = 'browser', usado pra quando o programa estiver operacional e user-friendly


In [120]:
def ColetarBaseDeDados(dia, mes, ano = 2020):
    global df
    if mes < 10 and dia < 10:
        nome = f"DT_PAINEL_COVIDBR_{ano}0{mes}0{dia}.xlsx"
        nome1 = f"DT1_PAINEL_COVIDBR_{ano}0{mes}0{dia}.xlsx"
        nome2 = f"HIST_PAINEL_COVIDBR_{ano}0{mes}0{dia}.xlsx"
    elif mes < 10:
        nome = f"DT_PAINEL_COVIDBR_{ano}0{mes}{dia}.xlsx"
        nome1 = f"DT1_PAINEL_COVIDBR_{ano}0{mes}{dia}.xlsx"
        nome2 = f"HIST_PAINEL_COVIDBR_{ano}0{mes}{dia}.xlsx"
    elif dia < 10:
        nome = f"DT_PAINEL_COVIDBR_{ano}{mes}0{dia}.xlsx"
        nome1 = f"DT1_PAINEL_COVIDBR_{ano}{mes}0{dia}.xlsx"
        nome2 = f"HIST_PAINEL_COVIDBR_{ano}{mes}0{dia}.xlsx"
    else:
        nome = f"DT_PAINEL_COVIDBR_{ano}{mes}{dia}.xlsx"
        nome1 = f"DT1_PAINEL_COVIDBR_{ano}{mes}{dia}.xlsx"
        nome2 = f"HIST_PAINEL_COVIDBR_{ano}{mes}{dia}.xlsx"

    #importando a base de dados
    try:
        df = pd.read_excel(nome2)
    except:
        try:
            df = pd.read_excel(nome)
        except:
            try:
                df = pd.read_excel(nome1)
            except:
                try:
                    ultimo_arquivo = max(glob.glob(f"{str(pathlib.Path().absolute())}/*.xlsx"), key=os.path.getctime)
                    df = pd.read_excel(ultimo_arquivo)
                    print(f"Por inconsistências do Ministério de Saúde na hora de nomear o arquivo, localizou-se a planilha mais recentemente adicionada na pasta:\n{ultimo_arquivo}")
                except:
                    print("Não foi possível localizar o arquivo do dia solicitado.")

    #limpando a base de dados
    df.drop(columns = ['coduf', 'codmun', 'codRegiaoSaude','nomeRegiaoSaude','Recuperadosnovos','emAcompanhamentoNovos'], inplace = True)
    df.rename({'semanaEpi':'semana_epi','populacaoTCU2019':'populacao','casosAcumulado':'casos','obitosAcumulado':'obitos'}, axis = 1, inplace = True)
    df.dropna(subset = ['municipio'], axis = 0, inplace = True)
    df['data'] = pd.to_datetime(df['data'])
    print("Dados obtidos com sucesso!")

In [119]:
#funcao que permite a seleção da cidade e gera o gráfico
def EstatisticasCidade():
    global df
    regiao = ""
    uf = ""
    while True:
        print("Qual a cidade cujo histórico você deseja checar? Lembre-se dos acentos e de usar letra maiúscula!")
        cidade = input()
        if " " in cidade:
            cidade = str(cidade)
        else:
            cidade = str(cidade).capitalize()

        df_cidade = df.loc[df['municipio'] == cidade]

        if cidade.capitalize() == "Sair":
            clear_output()
            print("Usuário escolheu sair do gerador de gráficos.")
            break

        elif df_cidade.empty == True:
            clear_output()
            print(f"{cidade} não foi encontrada.")
            print("Certifique-se que você escreveu o nome da cidade corretamente e que ela tenha pelo menos um caso de Coronavírus!")
            print(f"Se desejar sair da caixa de diálogo, escreva 'sair'\n")
            pass
        else:
            #controle de erros (pra caso tenha mais de uma cidade com o mesmo nome)
            if len(set(df_cidade['regiao'].values.tolist())) > 1:
                while True:
                    clear_output()
                    print(f"Eita, parece que existem várias cidades com o nome {cidade}.\nQual a região da cidade que você está procurando?")
                    regiao = str(input()).capitalize()
                    df_cidade = df_cidade.loc[df_cidade['regiao'] == regiao]

                    if regiao == "Sair":
                        break
                    elif df_cidade.empty == True:
                        print(f"{cidade} não foi encontrada na região {regiao}.")
                        print("Certifique-se que você escreveu o nome da região corretamente!")
                        print(f"Se desejar sair da caixa de diálogo, escreva 'sair'\n")
                        pass
                    #vai que existem duas cidades com o mesmo nome na mesma região
                    elif len(set(df_cidade['estado'].values.tolist())) > 1:
                        while True:
                            clear_output()
                            print(f"Eita, parece que existem várias cidades com o nome {cidade} na regiao {regiao}")
                            print("Qual a sigla da UF da cidade que você está procurando?")
                            print(f"Se desejar sair da caixa de diálogo, escreva 'sair'\n")
                            uf = str(input()).upper()
                            df_cidade = df_cidade.loc[df_cidade['estado'] == uf]
                            if uf == "Sair":
                                break
                            elif df_cidade.empty == True:
                                print(f"{cidade} não foi encontrada na UF {uf}.")
                                print("Certifique-se que você escreveu a sigla da UF corretamente!")
                                print(f"Se desejar sair da caixa de diálogo, escreva 'sair'\n")
                            elif len(set(df_cidade['estado'].values.tolist())) > 1:
                                print(f"Em {uf}, há mais de uma cidade com o nome {cidade}")
                                print("Infelizmente, com os dados do Ministério da Saúde, esse programa não consegue gerar gráficos nesse caso específico")
                                uf = "Sair"
                                break
                            else:
                                break
                    else:
                        break
            if regiao == "Sair" or uf == "Sair":
                clear_output()
                print("Usuário escolheu sair do gerador de gráficos.")
                print("Obrigado por usar o programa")
                break

            #limpando a tela
            clear_output()

            #capturando a UF da cidade
            uf = str(df_cidade['estado'].head(1))

            #capturando e formatando todas as datas desde que a cidade registrou o 1º caso
            datas = df_cidade['data'].dt.strftime('%d/%m/%Y').values.tolist()
            #capturando o número de dias que se passaram desde o 1º caso
            num_dias = len(datas)
            #fazendo uma array [1,2…,numero de dias]
            eixo_x = np.arange(num_dias)
            #capturando a data atual
            data_atual = datas[num_dias - 1]

            #capturando o número atual acumulado de casos e mortes da cidade
            num_de_casos = int(df_cidade['casos'].tail(1))
            num_de_mortes = int(df_cidade['obitos'].tail(1))

            #capturando o número inicial de casos (referente à primeira data disponível) e criando as funções das retas auxiliares
            num_inicial_de_casos = int(df_cidade['casos'].head(1))
            casos_uma_semana = [round(num_inicial_de_casos * (2 ** (x/7)),0) for x in eixo_x]
            casos_duas_semanas  = [round(num_inicial_de_casos * (2 ** (x/14)),0) for x in eixo_x]
            casos_dez_dias  = [round(num_inicial_de_casos * (2 ** (x/10)),0) for x in eixo_x]

            if num_de_mortes > 0:
                df_cidade_prim_morte = df_cidade.loc[df['obitos'] > 0]
                num_inicial_de_obitos = int(df_cidade_prim_morte['obitos'].head(1))
                obitos_uma_semana = [round(num_inicial_de_obitos * (2 ** (x/7)),0) for x in eixo_x]
                obitos_duas_semanas  = [round(num_inicial_de_obitos * (2 ** (x/14)),0) for x in eixo_x]
                obitos_dez_dias  = [round(num_inicial_de_obitos * (2 ** (x/10)),0) for x in eixo_x]

            #plottando os gráficos
            fig = make_subplots(
                rows = 2, cols = 1, 
                subplot_titles= (f"<b>Número de Casos Acumulados</b>",f"<b>Número de Óbitos Acumulados</b>"),
                vertical_spacing = 0.18
            )

            #renomeando as colunas para os nomes aparecerem bonitinhos no iplot
            df_cidade.rename({'casos':'Casos','obitos':'Óbitos','data':'Data'}, axis = 1, inplace = True)
            
            #primeiro gráfico
            fig.add_trace(
                go.Scatter(x = df_cidade['Data'], y= df_cidade['Casos'], 
                name = "Casos", line = dict(color = 'red',width = 3.5), 
                fill = 'tozeroy', fillcolor = 'rgba(255,0,0,0.1)'), 
                row = 1, col = 1
            )
            #criando as linhas de cenários
            fig.add_trace(
                go.Scatter(x = df_cidade['Data'], y= casos_uma_semana, 
                name = "Caso dobrasse a cada semana", line = dict(color = 'black',width = 3, dash = 'dot')), #formatando o nome da linha e o estilo da linha
                row = 1, col = 1
            )
            fig.add_trace(
                go.Scatter(x = df_cidade['Data'], y= casos_dez_dias, 
                name = "Caso dobrasse a cada dez dias", line = dict(color = 'purple',width = 3, dash = 'dot')), 
                row = 1, col = 1
            )
            fig.add_trace(
                go.Scatter(x = df_cidade['Data'], y= casos_duas_semanas, 
                name = "Caso dobrasse a cada duas semanas", line = dict(color = 'green',width = 3, dash = 'dot')), 
                row = 1, col = 1
            )

            #segundo gráfico
            fig.add_trace(
                go.Scatter(x = df_cidade['Data'], y= df_cidade['Óbitos'], 
                name = "Óbitos", line = dict(color = 'blue',width = 3.5), 
                fill = 'tozeroy', fillcolor = 'rgba(0,0,255,0.1)'), 
                row = 2, col = 1
            )
            #criando as linhas de cenários
            if num_de_mortes > 0:
                fig.add_trace(
                    go.Scatter(x = df_cidade_prim_morte['data'], y = obitos_uma_semana,  
                    name = "Caso dobrasse a cada semana", showlegend = False, line = dict(color = 'black', width = 3, dash = 'dot')), 
                    row = 2, col = 1
                )
                fig.add_trace(
                    go.Scatter(x = df_cidade_prim_morte['data'], y= obitos_dez_dias, 
                    name = "Caso dobrasse a cada dez dias", showlegend = False, line = dict(color = 'purple',width = 3, dash = 'dot')), 
                    row = 2, col = 1
                )
                fig.add_trace(
                    go.Scatter(x = df_cidade_prim_morte['data'], y= obitos_duas_semanas, 
                    name = "Caso dobrasse a cada duas semanas",showlegend = False, line = dict(color = 'green',width = 3, dash = 'dot')), 
                    row = 2, col = 1
                )

            #nomeando e ajustando a altura dos eixos y
            fig.update_yaxes(title_text = "Casos", row = 1, col = 1, range = [num_inicial_de_casos, num_de_casos + num_de_casos/100])
            fig.update_yaxes(title_text = "Mortes", row = 2, col = 1, range = [0, num_de_mortes + num_de_mortes/50])

            #criando os botões de seleção temporal
            fig.update_xaxes(
                range = [datas[0],datas[num_dias - 1]],
                rangeselector=dict(
                    buttons=list([
                        dict(count=7, label="Última Semana", step="day", stepmode="backward"),
                        dict(count=14, label="Última Duas Semanas", step="day", stepmode="backward"),
                        dict(count=1, label="Último Mês", step="month", stepmode="backward"),
                        dict(count=2, label="Último Dois Meses", step="month", stepmode="backward"),
                        dict(label = "Desde o 1º Caso", step="all")
                    ])
                )
            )

            #ajustando a posição dos títulos de cada gráfico
            fig.layout.annotations[0].update(x=0.85, y = 1.02)
            fig.layout.annotations[1].update(x=0.848, y = 0.43)

            #ajustando a posição da legenda, dando título à figura e ajustando suas dimensões
            fig.update_layout(
                template = "plotly_white",
                legend_orientation = 'h', legend = dict(x = 0,y = 0.53), 
                title_text = f"<b>Painel do Coronavírus relativo à cidade de {cidade}-{uf} ({data_atual})</b>",
                height = 800, width = 1000,
                margin=dict(l=30, r=30, t=100, b=20)
            )
            fig.show()

            #calculando alguns indicadores disponiveis no site do ministério da saúde
            print(f"Indicadores de Contágio em {cidade}:")
            print(f"\tNovos Casos: {int(df_cidade['Casos'].tail(1)) - int(df_cidade['Casos'].tail(2).head(1))} \tCoeficiente de Incidência: {round(num_de_casos*100000/int(df_cidade['populacao'].tail(1)),2)}")
            print(f"\nIndicadores de Mortalidade em {cidade}:")
            print(f"\tNovos Óbitos: {int(df_cidade['Óbitos'].tail(1)) - int(df_cidade['Óbitos'].tail(2).head(1))} \tCoeficiente de Mortalidade: {round(num_de_mortes*100000/int(df_cidade['populacao'].tail(1)),2)} \tLetalidade: {round(num_de_mortes/num_de_casos*100,2)}%")
            print("\nPara mais informações sobre interpretações, usos e limitações dos indicadores, acesse o site do Ministério da Saúde: https://covid.saude.gov.br")
            break

**AVISOS IMPORTANTES**

Para o programa funcionar corretamente:

1. É necessário baixar manualmente o arquivo de Excel referente ao dia atual, disponível no site https://covid.saude.gov.br;
2. Após baixar o arquivo, certifique-se de que ele está na ***mesma pasta*** em que esse script está armazenado.

Na função "ColetarBaseDeDados", insira como argumentos, respectivamente, o dia e o mês atual (ou referente ao dia do arquivo baixado do site do Ministério da Saúde). 

In [124]:
#primeiro argumento: dia
#segundo argumento: mês
#exemplo: coletar_base_de_dados(11, 5)
#para rodar a célula, pressione Shift+Enter
ColetarBaseDeDados(20, 5)

Dados obtidos com sucesso!


In [125]:
#para rodar a célula, pressione Shift+Enter e siga as intruções para gerar o gráfico
EstatisticasCidade()

Indicadores de Contágio em Brasília:
	Novos Casos: 224 	Coeficiente de Incidência: 113.29

Indicadores de Mortalidade em Brasília:
	Novos Óbitos: 3 	Coeficiente de Mortalidade: 1.69 	Letalidade: 1.49%

Para mais informações sobre interpretações, usos e limitações dos indicadores, acesse o site do Ministério da Saúde: https://covid.saude.gov.br


**Obrigado por usar o programa. Qualquer sugestão ou aprimoramento é muito bem-vinda ;)**